In [1]:
import os

import requests
import pandas as pd
idx = pd.IndexSlice
datos_cnbv = 'D:/datos/CNBV/operativos'

# Descarga datos

In [47]:
# Descarga
for y in [2019, 2020]:
    for m in range(1, 13):
        periodo = f'{y}{m:02}'
        bm_file = f'{datos_cnbv}/BM_Operativa_{periodo}.xlsx'
        if not os.path.exists(bm_file):
            url = f'https://portafolioinfo.cnbv.gob.mx/PortafolioInformacion/BM_Operativa_{periodo}.xlsx'
            header = {}
            resp = requests.get(url, stream=True)
            if resp.status_code == 200:
                with open(bm_file, 'wb') as excelfile:
                    excelfile.write(resp.content)
            else:
                print('problema con', periodo)

problema con 202010
problema con 202011
problema con 202012


In [50]:
transacciones = ['Número de Transacciones en TPV', 'Número de Transacciones en Cajeros Automáticos']
for y in [2019, 2020]:
    for m in range(1, 13):
        periodo = f'{y}{m:02}'
        bm_file = f'{datos_cnbv}/BM_Operativa_{periodo}.xlsx'
        trans_file = f'datos/transacciones_cnbv_{periodo}.csv'
        if os.path.exists(bm_file) & (not os.path.exists(trans_file)):
            periodo = f'{y}{m:02}'
            cnbv = pd.read_excel(f'{datos_cnbv}/BM_Operativa_{periodo}.xlsx', sheet_name='Hoja1')
            cnbv_trans = cnbv.query('dl_producto_financiero==@transacciones & cve_inegi.notnull()==True', engine='python').copy()
            cnbv_trans['cve_mun'] = cnbv_trans.cve_inegi.astype(str).str[3:8].astype(int)
            cnbv_trans['cve_ent'] = cnbv_trans.cve_inegi.astype(str).str[3:5].astype(int)
            cnbv_trans_group = cnbv_trans.groupby(['cve_periodo', 'cve_ent', 'cve_mun', 'dl_producto_financiero'], as_index=False)['dat_num_total'].sum()
            cnbv_trans_group.to_csv(trans_file, index=False)
        else:
            print('problema con', periodo)

problema con 202010
problema con 202011
problema con 202012


## Conforma base única

In [55]:
df_trans = pd.concat([pd.read_csv(f'datos/transacciones_cnbv_2020{m:02}.csv').assign(year=2020, mes=m) for m in range(1, 9)])
df_trans.to_csv('transacciones_cnbv_2020.csv', index=False)

# Crecimiento

In [3]:
df_trans = pd.read_csv('datos/transacciones_cnbv_2020.csv')
df_trans['crec_acum'] = df_trans.groupby(['year', 'cve_ent', 'cve_mun', 'dl_producto_financiero'])['dat_num_total'].transform(lambda s: s.pct_change().add(1).cumprod().subtract(1).mul(100))
df_trans.set_index(['cve_ent', 'cve_mun', 'mes', 'dl_producto_financiero'], inplace=True)
df_trans.sort_index(inplace=True)
df_trans_crec_acum = df_trans['crec_acum'].unstack('dl_producto_financiero').dropna()\
    .rename(columns=lambda x: x.lower().replace(' ', '_').replace('ú', 'u').replace('á', 'a'))
df_trans_crec_acum.to_csv('datos/crecimiento_acumulado_transacciones_cnbv_202008.csv', encoding='utf8')
df_trans_crec_acum.head()

dl_producto_financiero  numero_de_transacciones_en_cajeros_automaticos  \
cve_ent cve_mun mes                                                      
1       1001    2                                           -14.653296   
                3                                           -17.019599   
                4                                           -35.145484   
                5                                           -27.079090   
                6                                           -27.947189   

dl_producto_financiero  numero_de_transacciones_en_tpv  
cve_ent cve_mun mes                                     
1       1001    2                            -2.778147  
                3                            -5.171215  
                4                           -27.769826  
                5                           -24.810371  
                6                           -12.945142